In [1]:
import asyncio
from typing import Annotated

from semantic_kernel.agents import ChatCompletionAgent
from semantic_kernel.connectors.ai.function_choice_behavior import FunctionChoiceBehavior
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion
from semantic_kernel.contents.chat_history import ChatHistory
from semantic_kernel.contents.utils.author_role import AuthorRole
from semantic_kernel.functions.kernel_function_decorator import kernel_function
from semantic_kernel.kernel import Kernel

In [11]:
# This sample allows for a streaming response verus a non-streaming response
streaming = False

# Define the agent name and instructions
HOST_NAME = "Host"
HOST_INSTRUCTIONS = "Answer questions about the menu."


# Define a sample plugin for the sample
class MenuPlugin:
    """A sample Menu Plugin used for the concept sample."""

    @kernel_function(description="Provides a list of specials from the menu.")
    def get_specials(self) -> Annotated[str, "Returns the specials from the menu."]:
        return """
        Special Soup: Clam Chowder
        Special Salad: Cobb Salad
        Special Drink: Chai Tea
        """

    @kernel_function(description="Provides the price of the requested menu item.")
    def get_item_price(self, 
        menu_item: Annotated[str, "The name of the menu item."]) -> Annotated[str, "Returns the price of the menu item."]:
        if menu_item == "Clam Chowder":
            return 9.99
        elif menu_item == "Cobb Salad":
            return 8.50
        elif menu_item == "Chai Tea":
            return 3.55
        else:
            return "Item not found"

In [12]:
# A helper method to invoke the agent with the user input
async def invoke_agent(agent: ChatCompletionAgent, input: str, chat: ChatHistory) -> None:
    """Invoke the agent with the user input."""
    chat.add_user_message(input)

    print(f"# {AuthorRole.USER}: '{input}'")

    if streaming:
        contents = []
        content_name = ""
        async for content in agent.invoke_stream(chat):
            content_name = content.name
            contents.append(content)
        message_content = "".join([content.content for content in contents])
        print(f"# {content.role} - {content_name or '*'}: '{message_content}'")
        chat.add_assistant_message(message_content)
    else:
        async for content in agent.invoke(chat):
            print(f"# {content.role} - {content.name or '*'}: '{content.content}'")
        chat.add_message(content)


In [14]:
# Create the instance of the Kernel
kernel = Kernel()

service_id = "agent"
kernel.add_service(AzureChatCompletion(service_id=service_id))

settings = kernel.get_prompt_execution_settings_from_service_id(service_id=service_id)
# Configure the function choice behavior to auto invoke kernel functions
settings.function_choice_behavior = FunctionChoiceBehavior.Auto()

kernel.add_plugin(MenuPlugin(), plugin_name="menu")

# Create the agent
agent = ChatCompletionAgent(
    service_id="agent", kernel=kernel, name=HOST_NAME, instructions=HOST_INSTRUCTIONS, execution_settings=settings
)

# Define the chat history
chat = ChatHistory()

# Respond to user input
await invoke_agent(agent, "Hello", chat)
await invoke_agent(agent, "What is the special soup?", chat)
await invoke_agent(agent, "What is the special drink and what is its price?", chat)
await invoke_agent(agent, "How about the salad?", chat)
await invoke_agent(agent, "How much is the salad?", chat)
await invoke_agent(agent, "Thank you", chat)


# AuthorRole.USER: 'Hello'
# AuthorRole.ASSISTANT - Host: 'Hi there! How can I assist you today?'
# AuthorRole.USER: 'What is the special soup?'
# AuthorRole.ASSISTANT - Host: 'The special soup today is Clam Chowder. Would you like to know anything else about the menu?'
# AuthorRole.USER: 'What is the special drink and what is its price?'
# AuthorRole.ASSISTANT - Host: 'The special drink today is Chai Tea, and its price is $3.55. Is there anything else you'd like to know?'
# AuthorRole.USER: 'How about the salad?'
# AuthorRole.ASSISTANT - Host: 'The special salad today is the Cobb Salad. Would you like to know its price or more details about it?'
# AuthorRole.USER: 'How much is the salad?'
# AuthorRole.ASSISTANT - Host: 'The Cobb Salad is priced at $8.50. Is there anything else you need?'
# AuthorRole.USER: 'Thank you'
# AuthorRole.ASSISTANT - Host: 'You're welcome! If you have any more questions, feel free to ask. Have a great day!'
